In [5]:
import numpy as np

In [6]:
def shard_semantic3d(
    xyzrgb_path: str,
    labels_path: str,
    shard_size: int = 1_000_000,
    out_prefix: str = "shard_"
):
    labels_all = np.loadtxt(labels_path, dtype=np.int32)
    shard_points = []
    shard_labels = []
    shard_idx = 0
    i = 0

    with open(xyzrgb_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) not in (4, 6, 7):
                continue
            vals = list(map(float, parts))

            if len(vals) == 7:
                X, Y, Z, intensity, R, G, B = vals
            elif len(vals) == 6:
                X, Y, Z, R, G, B = vals
                intensity = 0.0
            elif len(vals) == 4:
                X, Y, Z, intensity = vals
                R = G = B = 0.0

            if i >= len(labels_all):
                break

            shard_points.append([X, Y, Z, R, G, B, intensity])
            shard_labels.append(labels_all[i])
            i += 1

            if len(shard_points) >= shard_size:
                points_arr = np.array(shard_points, dtype=np.float32)
                labels_arr = np.array(shard_labels, dtype=np.int32)
                np.savez(f"{out_prefix}{shard_idx}.npz",
                         points=points_arr, labels=labels_arr)
                shard_points = []
                shard_labels = []
                shard_idx += 1

    if shard_points:
        points_arr = np.array(shard_points, dtype=np.float32)
        labels_arr = np.array(shard_labels, dtype=np.int32)
        np.savez(f"{out_prefix}{shard_idx}.npz",
                 points=points_arr, labels=labels_arr)


In [7]:
shard_semantic3d(
    "bildstein_station3_xyz_intensity_rgb.txt",
    "bildstein_station3_xyz_intensity_rgb.labels"
)

### Объединение всех шардов в финальный файл

In [8]:
import glob
import os

def merge_shards(prefix: str = "shard_",
                 out_path: str = "semantic3d_merged.npz"):

    shard_files = sorted(glob.glob(f"{prefix}*.npz"))
    if not shard_files:
        raise FileNotFoundError(f"Не найдено ни одного файла с префиксом '{prefix}'")

    points_list = []
    labels_list = []

    for fname in shard_files:
        print(f"Загружаю {fname} ...")
        data = np.load(fname)
        points_list.append(data["points"])
        labels_list.append(data["labels"])

    points = np.concatenate(points_list, axis=0)
    labels = np.concatenate(labels_list, axis=0)

    print("Итоговая форма points:", points.shape)
    print("Итоговая форма labels:", labels.shape)

    np.savez_compressed(out_path, points=points, labels=labels)
    print(f"Сохранено в {out_path}")

    return points.shape, labels.shape


In [9]:
points_shape, labels_shape = merge_shards(
    prefix="shard_",                    # если файлы shard_0.npz, shard_1.npz, ...
    out_path="semantic3d_full.npz"      # единый файл
)
print(points_shape, labels_shape)


Загружаю shard_0.npz ...
Загружаю shard_1.npz ...
Загружаю shard_10.npz ...
Загружаю shard_11.npz ...
Загружаю shard_12.npz ...
Загружаю shard_13.npz ...
Загружаю shard_14.npz ...
Загружаю shard_15.npz ...
Загружаю shard_16.npz ...
Загружаю shard_17.npz ...
Загружаю shard_18.npz ...
Загружаю shard_19.npz ...
Загружаю shard_2.npz ...
Загружаю shard_20.npz ...
Загружаю shard_21.npz ...
Загружаю shard_22.npz ...
Загружаю shard_23.npz ...
Загружаю shard_3.npz ...
Загружаю shard_4.npz ...
Загружаю shard_5.npz ...
Загружаю shard_6.npz ...
Загружаю shard_7.npz ...
Загружаю shard_8.npz ...
Загружаю shard_9.npz ...
Итоговая форма points: (23995481, 7)
Итоговая форма labels: (23995481,)
Сохранено в semantic3d_full.npz
(23995481, 7) (23995481,)


### Первые 5 строк

In [10]:
data = np.load("semantic3d_full.npz")
points = data["points"]
labels = data["labels"]

print("Первые 5 строк points:\n", points[:5])
print("Первые 5 меток:", labels[:5])


Первые 5 строк points:
 [[-3.0000e-03 -3.1752e+01 -2.6180e+00  1.3000e+02  1.4600e+02  1.6200e+02
  -1.3400e+02]
 [-1.0000e-03 -3.1756e+01 -2.8070e+00  1.4200e+02  1.5700e+02  1.7800e+02
  -8.9000e+01]
 [-2.0000e-03 -3.1759e+01 -2.8990e+00  1.4400e+02  1.5700e+02  1.7600e+02
  -2.8100e+02]
 [-2.0000e-03 -3.1759e+01 -2.9020e+00  1.4300e+02  1.5600e+02  1.7500e+02
  -1.6200e+02]
 [-3.0000e-03 -3.1759e+01 -2.9020e+00  1.4300e+02  1.5600e+02  1.7500e+02
  -1.1300e+02]]
Первые 5 меток: [0 0 0 0 0]
